In [1]:
print('ok')

ok


In [2]:
%pwd

'c:\\Users\\Sonjoy Dey\\Desktop\\SD\\Wix-buddy\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Sonjoy Dey\\Desktop\\SD\\Wix-buddy'

In [5]:
import json
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
def load_json_files(directory):
    documents = []

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                    content = json.dumps(data, indent=2)  # Customize as needed

                    doc = Document(
                        page_content=content,
                        metadata={"source": filename}
                    )
                    documents.append(doc)
            except json.JSONDecodeError as e:
                print(f"Skipping {filename} — JSON decode error: {e}")
            except Exception as e:
                print(f"Skipping {filename} — Unexpected error: {e}")

    return documents


In [12]:
extracted_data = load_json_files(directory='Data/')

In [14]:
print(f"Extracted {len(extracted_data)} documents from JSON files.")

Extracted 1 documents from JSON files.


In [19]:
# Optional: Split long documents into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(documents)

In [20]:
text_chunks=split_documents(extracted_data)

In [21]:
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 66296


In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

In [26]:
def download_huggingface_model_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [27]:
embeddings = download_huggingface_model_embedding()

In [29]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [32]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "wixbuddy"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "wixbuddy",
    "metric": "cosine",
    "host": "wixbuddy-uor1e5c.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [33]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [34]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [35]:
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [37]:
retrieved_docs = retriever.invoke("Some bangaldeshi university") 

In [38]:
retrieved_docs

[Document(id='40992057-9240-41ad-8e24-4218a558b3d1', metadata={'source': 'uni_dataset.json'}, page_content=']\n    },\n    "divisions": {\n      "Economics": [\n        "Accountancy",\n        "Economics",\n        "Information Management",\n        "Management"\n      ],\n      "Engineering": [\n        "Civil Engineering",\n        "Engineering",\n        "Mechanical Engineering"\n      ],\n      "Islamic Studies": [\n        "Islamic Studies"\n      ],\n      "Law": [\n        "Law"\n      ],\n      "Teacher Training and Education": [\n        "Biology",\n        "Curriculum",\n        "Educational and Student Counselling",\n        "Educational Sciences",\n        "Educational Technology",\n        "History",\n        "Mathematics Education",\n        "Natural Sciences",\n        "Physics",\n        "Primary Education",\n        "Teacher Training"\n      ],\n      "Graduate Studies": [\n        "Educational Administration",\n        "Management",\n        "Science Education"\n     

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.4, 
    max_tokens=500, 

)

In [41]:
# Prepare your message
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

# Invoke the LLM
ai_msg = llm.invoke(messages)

# Print the result
print(ai_msg.content)

J'adore la programmation.


In [71]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an intelligent assistant for a study consultancy system. "
    "Use the retrieved context below to answer the user's question as accurately and concisely as possible. "
    "If the context does not provide enough information, indicate that you're searching the web to find the answer. "
    "Keep responses to a maximum of three sentences and prioritize clarity."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [72]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [77]:
response = rag_chain.invoke({"input": "Tell me about Chittagong University offered course."})
print(response["answer"])

Chittagong University offers courses in Arts and Humanities, Business and Economics, and Science and Engineering. The Arts and Humanities include fields like Literature, Philosophy, and Political Sciences. Science and Engineering programs cover Computer Science, Geography, and Psychology.


response = rag_chain.invoke({"input": "Tell me about Chittagong University offered course."})
print(response["answer"])